<a href="https://qworld.net" target="_blank" align="left"><img src="../qworld/images/header.jpg"  align="left"></a>
$ \renewcommand{\bra}[1]{\langle #1|} $
$ \renewcommand{\ket}[1]{|#1\rangle} $
$ \renewcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \renewcommand{\dot}[2]{ #1 \cdot #2} $
$ \renewcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \renewcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \renewcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \renewcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \renewcommand{\mypar}[1]{\left( #1 \right)} $
$ \renewcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \renewcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \renewcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \renewcommand{\onehalf}{\frac{1}{2}} $
$ \renewcommand{\donehalf}{\dfrac{1}{2}} $
$ \renewcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \renewcommand{\vzero}{\myvector{1\\0}} $
$ \renewcommand{\vone}{\myvector{0\\1}} $
$ \renewcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \renewcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \renewcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \renewcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \renewcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \renewcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \renewcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \renewcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \renewcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \renewcommand{\pstate}[1]{ \lceil #1 \rfloor } $
$ \renewcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \renewcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \renewcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \renewcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \renewcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

<font style="font-size:28px;" align="left"><b>Quantum Tomography  </b></font>
<br>
_prepared by Abuzer Yakaryilmaz_
<br><br>
[<img src="../qworld/images/watch_lecture.jpg" align="left">](https://youtu.be/mIEiWCJ6R58)
<br><br><br>

We study a simplified version of quantum tomography here. 

It is similar to learn the bias of a coin by collecting statistics from tossing this coin many times. But, only making measurement may not be enough to make a good guess.

Suppose that you are given 1000 copies of a qubit and your task is to learn the state of this qubit. We use a python class called "unknown_qubit" for doing our quantum experiments. 

Please run the following cell before continuing.

In [1]:
# class unknown_qubit
#   available_qubit = 1000 -> you get at most 1000 qubit copies
#   get_qubits(number_of_qubits) -> you get the specified number of qubits for your experiment
#   measure_qubits() -> your qubits are measured and the result is returned as a dictionary variable
#                    -> after measurement, these qubits are destroyed
#   rotate_qubits(angle) -> your qubits are rotated with the specified angle in radian
#   compare_my_guess(my_angle) -> your guess in radian is compared with the real angle

from random import randrange
from math import pi
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
class unknown_qubit:   
    def __init__(self):
        self.__theta = randrange(18000)/18000*pi        
        self.__available_qubits = 1000
        self.__active_qubits = 0
        print(self.__available_qubits,"qubits are created")
    
    def get_qubits(self,number_of_qubits=None):
        if number_of_qubits is None or isinstance(number_of_qubits,int) is False or number_of_qubits < 1:
            print()
            print("ERROR: the method 'get_qubits' takes the number of qubit(s) as a positive integer, i.e., get_qubits(100)")
        elif number_of_qubits <= self.__available_qubits:
            self.__qc = QuantumCircuit(1,1)
            self.__qc.ry(2 * self.__theta,0)
            self.__active_qubits = number_of_qubits
            self.__available_qubits = self.__available_qubits - self.__active_qubits
            print()
            print("You have",number_of_qubits,"active qubits that are set to (cos(theta),sin(theta))")
            self.available_qubits()
        else:
            print()
            print("WARNING: you requested",number_of_qubits,"qubits, but there is not enough available qubits!")
            self.available_qubits()
            
    def measure_qubits(self):    
        if self.__active_qubits > 0:            
            self.__qc.measure(0,0)
            job = execute(self.__qc,Aer.get_backend('qasm_simulator'),shots=self.__active_qubits)
            counts = job.result().get_counts(self.__qc)
            print()
            print("your",self.__active_qubits,"qubits are measured")
            print("counts = ",counts)
            self.__active_qubits = 0
            return counts
        else:
            print()
            print("WARNING: there is no active qubits -- you might first execute 'get_qubits()' method")
            self.available_qubits()
            
    def rotate_qubits(self,angle=None):
        if angle is None or (isinstance(angle,float) is False and isinstance(angle,int) is False):
            print()
            print("ERROR: the method 'rotate_qubits' takes a real-valued angle in radian as its parameter, i.e., rotate_qubits(1.2121)")
        elif self.__active_qubits > 0:
            self.__qc.ry(2 * angle,0)
            print()
            print("your active qubits are rotated by angle",angle,"in radian")
        else:
            print()
            print("WARNING: there is no active qubits -- you might first execute 'get_qubits()' method")
            self.available_qubits()    
    
    def compare_my_guess(self,my_angle):
        if my_angle is None or (isinstance(my_angle,float) is False and isinstance(my_angle,int) is False):
            print("ERROR: the method 'compare_my_guess' takes a real-valued angle in radian as your guessed angle, i.e., compare_my_guess(1.2121)")
        else:
            self.__available_qubits = 0
            diff = abs(my_angle-self.__theta)
            print()
            print(self.__theta,"is the original",)
            print(my_angle,"is your guess")
            print("the angle difference between the original theta and your guess is",diff/pi*180,"degree")
            print("-->the number of available qubits is (set to) zero, and so you cannot make any further experiment")

    def available_qubits(self):
        print("--> the number of available unused qubit(s) is",self.__available_qubits)              

class unknown_qubit:
    
    available_qubit = 1000 -> you get at most 1000 qubit copies
    get_qubits(number_of_qubits) -> you get the specified number of qubits for your experiment
    measure_qubits() -> your qubits are measured and the result is returned as a dictionary variable
                     -> after measurement, these qubits are destroyed
    rotate_qubits(angle) -> your qubits are rotated with the specified angle in radian
    compare_my_guess(my_angle) -> your guess in radian is compared with the real angle

<h3> Task 1 </h3>

You are given 1000 copies of the identical qubits which are in the same quantum state lying in the first or second quadrant of the unit circle. 

This quantum state is represented by an angle $ \theta \in [0,\pi) $, and your task is to guess this angle.

You use the class __unknown_qubit__ and its methods for your experiments. 

_Remark that the measurement outcomes of the quantum states with angles $ \pi \over 3 $ and $ 2 \pi \over 3 $ are identical even though they are different quantum states. Therefore, getting 1000 qubits and then measuring them does not guarantee the correct answer._

Test your solution at least ten times.

In [2]:
from math import pi, cos, sin, acos, asin

# an angle theta is randomly picked and it is fixed througout the experiment
my_experiment = unknown_qubit() 
#
# my_experiment.get_qubits(number_of_qubits)
# my_experiment.rotate_qubits(angle)
# my_experiment.measure_qubits()
# my_experiment.compare_my_guess(my_angle)
#

#
# your solution is here
#
my_experiment.get_qubits(900)
counts = my_experiment.measure_qubits()

noObservedZeros = 0
if '0' in counts:
    noObservedZeros = counts['0']

probability_of_observing_zeros = noObservedZeros / 900
cos_theta = probability_of_observing_zeros ** 0.5
theta = acos(cos_theta)

theta_first_candidate = theta
theta_second_candidate = pi-theta

print("The first candidate is ", theta_first_candidate, " in radian and ", theta_first_candidate*180/pi, " in degree")
print("The second candidate is ", theta_second_candidate, " in radian and ", theta_second_candidate*180/pi, " in degree")

1000 qubits are created

You have 900 active qubits that are set to (cos(theta),sin(theta))
--> the number of available unused qubit(s) is 100

your 900 qubits are measured
counts =  {'1': 610, '0': 290}
The first candidate is  0.9671523616037631  in radian and  55.413748466006076  in degree
The second candidate is  2.17444029198603  in radian and  124.58625153399394  in degree


In [3]:
my_experiment.get_qubits(100)
my_experiment.rotate_qubits(-1 * theta_first_candidate)

counts = my_experiment.measure_qubits()
noObservedZeros = 0
if '0' in counts:
    noObservedZeros = counts['0']

if noObservedZeros == 100:
    my_guess = theta_first_candidate
else:
    my_guess = theta_second_candidate
    
my_experiment.compare_my_guess(my_guess)


You have 100 active qubits that are set to (cos(theta),sin(theta))
--> the number of available unused qubit(s) is 0

your active qubits are rotated by angle -0.9671523616037631 in radian

your 100 qubits are measured
counts =  {'1': 91, '0': 9}

2.1762510443117296 is the original
2.17444029198603 is your guess
the angle difference between the original theta and your guess is 0.10374846600607288 degree
-->the number of available qubits is (set to) zero, and so you cannot make any further experiment


In [4]:
for i in range(10):
    #
    # your solution
    #
    print("Experiment", i + 1)
    print("___________\n")

    my_experiment = unknown_qubit()
    my_experiment.get_qubits(900)
    counts = my_experiment.measure_qubits()

    number_of_observed_zeros = 0
    if '0' in counts:
        number_of_observed_zeros = counts['0']

    probability_of_observing_zeros = number_of_observed_zeros / 900
    cos_theta = probability_of_observing_zeros ** 0.5
    theta = acos(cos_theta)

    theta_first_candidate = theta
    theta_second_candidate = pi-theta
    
    my_experiment.get_qubits(100)
    my_experiment.rotate_qubits(-1 * theta_first_candidate)

    counts = my_experiment.measure_qubits()
    number_of_observed_zeros = 0
    if '0' in counts:
        number_of_observed_zeros = counts['0']

    if number_of_observed_zeros == 100:
        my_guess = theta_first_candidate
    else:
        my_guess = theta_second_candidate

    my_experiment.compare_my_guess(my_guess)
    print()
    print()
    print()

Experiment 1
___________

1000 qubits are created

You have 900 active qubits that are set to (cos(theta),sin(theta))
--> the number of available unused qubit(s) is 100

your 900 qubits are measured
counts =  {'1': 804, '0': 96}

You have 100 active qubits that are set to (cos(theta),sin(theta))
--> the number of available unused qubit(s) is 0

your active qubits are rotated by angle -1.2380937093766728 in radian

your 100 qubits are measured
counts =  {'0': 100}

1.24651415177435 is the original
1.2380937093766728 is your guess
the angle difference between the original theta and your guess is 0.48245581101992613 degree
-->the number of available qubits is (set to) zero, and so you cannot make any further experiment



Experiment 2
___________

1000 qubits are created

You have 900 active qubits that are set to (cos(theta),sin(theta))
--> the number of available unused qubit(s) is 100

your 900 qubits are measured
counts =  {'1': 425, '0': 475}

You have 100 active qubits that are set 

[click for our solution](Q52_Quantum_Tomography_Solution.ipynb#task1)

<h3> Task 2 (extra) </h3>

You are given 1000 identical quantum systems with two qubits that are in states $ \myvector{\cos \theta_1 \\ \sin \theta_1} $ and $ \myvector{\cos \theta_2 \\ \sin \theta_2} $, where $ \theta_1,\theta_2 \in [0,\pi) $.

Your task is to guess the values of $ \theta_1 $ and $ \theta_2 $. 

Create a quantum circuit with two qubits. 

Randomly pick $\theta_1$ and $ \theta_2 $ and set the states of qubits respectively. (Do not use $ \theta_1 $ and $ \theta_2 $ except initializing the qubits.)

Do experiments (making measurements and/or applying basic quantum operators) with your circuit(s). You may create more than one circuit.

Assume that the total number of shots does not exceed 1000 throughout the whole experiment.

_Since you have two qubits, your measurement outcomes will be '00', '01', '10', and '11'._

In [ ]:
#
# your solution
#


<h3> Task 3 (Discussion) </h3>

If the angle in Task 1 is picked in range $ [0,2\pi) $, then can we determine its quadrant correctly?

<h3> Global phase </h3>

Suppose that we have a qubit and its state is either $ \ket{0} $ or $ -\ket{0} $.

Is there any sequence of one-qubit gates such that we can measure different results after applying them?

All one-qubit gates are $ 2 \times 2 $ matrices, and their application is represented by a single matrix: $ A_n \cdot \cdots \cdot A_2 \cdot A_1 = A $.

By linearity, if $ A \ket{0} = \ket{u} $, then $ A (- \ket{0}) = -\ket{u} $. Thus, after measurement, the probabilities of observing state $ \ket{0} $ and state $ \ket{1} $ are the same for $ \ket{u} $ and $ -\ket{u} $. Therefore, we cannot distinguish them.

Even though the states $ \ket{0} $ and $ -\ket{0} $ are different mathematically, they are assumed as identical from the physical point of view. 

The minus sign in front of $ -\ket{0} $ is called as a global phase.

In general, a global phase can be a complex number with magnitude 1.